## Package 

In [1]:
import tensorflow as tf
import tensorflow_datasets as tfds
import os
import re
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
print("슝=3")

슝=3


# 1. 데이터 전처리

## 데이터 가져오기

In [2]:
df = pd.read_csv("ChatbotData .csv")
df

,Q,A,label
0,12시 땡!,하루가 또 가네요.,0
1,1지망 학교 떨어졌어,위로해 드립니다.,0
2,3박4일 놀러가고 싶다,여행은 언제나 좋죠.,0
3,3박4일 정도 놀러가고 싶다,여행은 언제나 좋죠.,0
4,PPL 심하네,눈살이 찌푸려지죠.,0
...,...,...,...
11818,훔쳐보는 것도 눈치 보임.,티가 나니까 눈치가 보이는 거죠!,2
11819,훔쳐보는 것도 눈치 보임.,훔쳐보는 거 티나나봐요.,2
11820,흑기사 해주는 짝남.,설렜겠어요.,2
11821,힘든 연애 좋은 연애라는게 무슨 차이일까?,잘 헤어질 수 있는 사이 여부인 거 같아요.,2


### 1) 전처리 함수: 영어에 대한 전처리 함수

In [3]:
# 전처리 함수
def preprocess_sentence1(sentence):
  # 입력받은 sentence를 소문자로 변경하고 양쪽 공백을 제거
  sentence = sentence.lower().strip()  # 문장을 소문자로 변경하고 공백 제거

  # 단어와 구두점(punctuation) 사이의 거리를 만듭니다.
  # 예를 들어서 "I am a student." => "I am a student ."와 같이
  # student와 온점 사이에 거리를 만듭니다.
  sentence = re.sub(r"([?.!,])", r" \1 ", sentence)  # 구두점과 단어 사이에 공백 삽입
  sentence = re.sub(r'[" "]+', " ", sentence)  # 연속된 공백을 하나의 공백으로 변환

  # (a-z, A-Z, ".", "?", "!", ",")를 제외한 모든 문자를 공백인 ' '로 대체합니다.
  sentence = re.sub(r"[^a-zA-Z?.!,]+", " ", sentence)  # 허용된 문자 외 모든 문자를 공백으로 변환
  sentence = sentence.strip()  # 양쪽 공백 제거
  return sentence

print("슝=3")

슝=3


### 2) 전처리 함수2: 한국어에 대한 전처리 함수

In [4]:
# 전처리 함수
def preprocess_sentence2(sentence):
  # 입력받은 sentence를 소문자로 변경하고 양쪽 공백을 제거
    sentence = sentence.lower().strip()  # 문장을 소문자로 변경하고 공백 제거

  # 단어와 구두점(punctuation) 사이의 거리를 만듭니다.
  # 예를 들어서 "I am a student." => "I am a student ."와 같이
  # student와 온점 사이에 거리를 만듭니다.
    sentence = re.sub(r'[^\w\W\.\?!,]', "", sentence)  # 구두점과 단어 사이에 공백 삽입
    sentence = re.sub(r'[" "]+', " ", sentence)  # 연속된 공백을 하나의 공백으로 변환

  # (a-z, A-Z, ".", "?", "!", ",")를 제외한 모든 문자를 공백인 ' '로 대체합니다.
  # 허용된 문자 외 모든 문자를 공백으로 변환
    sentence = sentence.strip()  # 양쪽 공백 제거
    return sentence

print("슝=3")

슝=3


## 문제에서 사용
-questions과 answers에 대하여 전처리를 수행한다.

In [5]:
# 질문과 답변의 쌍인 데이터셋을 구성하기 위한 데이터 로드 함수
def load_conversations(df):
    inputs, outputs = [], []

    for line in df["A"]:
        inputs.append(preprocess_sentence2(line))
    
    for line in df['Q']: 
        outputs.append(preprocess_sentence2(line)) 

    return inputs, outputs
print("슝=3")

슝=3


In [6]:
questions, answers =load_conversations(df)
print('전체 샘플 수 :', len(questions))
print('전체 샘플 수 :', len(answers))
print('전처리 후의 23번째 질문 샘플: {}'.format(questions[22]))
print('전처리 후의 23번째 답변 샘플: {}'.format(answers[22]))

전체 샘플 수 : 11823
전체 샘플 수 : 11823
전처리 후의 23번째 질문 샘플: 가장 확실한 시간은 오늘이에요. 어제와 내일을 놓고 고민하느라 시간을 낭비하지 마세요.
전처리 후의 23번째 답변 샘플: 가장 확실한 건 뭘까?


# 2. SubwordTextEncoder 사용하기

## tokenizer 사용하기: 단어장 만들기
단어들에 index를 부여하기 위하여 tokenizer라는 단어장을 미리 만드는 과정이다.

In [7]:
import tensorflow_datasets as tfds
tokenizer = tfds.deprecated.text.SubwordTextEncoder.build_from_corpus(questions + answers, target_vocab_size=2**13)
START_TOKEN, END_TOKEN = [tokenizer.vocab_size], [tokenizer.vocab_size + 1]


In [8]:
START_TOKEN, END_TOKEN = [tokenizer.vocab_size], [tokenizer.vocab_size + 1]
# 문장의 시작과 끝에 해당하는 token을 추가한다.

In [9]:
print('START_TOKEN의 번호 :' ,[tokenizer.vocab_size])
print('END_TOKEN의 번호 :' ,[tokenizer.vocab_size + 1])
# 시작과 끝 token 번호

START_TOKEN의 번호 : [8170]
END_TOKEN의 번호 : [8171]


In [10]:
VOCAB_SIZE = tokenizer.vocab_size + 2 # 단어장 크기 더하기 2
print(VOCAB_SIZE)

8172


In [11]:
print('정수 인코딩 후의 21번째 질문 샘플: {}'.format(tokenizer.encode(questions[21])))
print('정수 인코딩 후의 21번째 답변 샘플: {}'.format(tokenizer.encode(answers[21])))

정수 인코딩 후의 21번째 질문 샘플: [2353, 7510, 5, 6273, 94, 7960]
정수 인코딩 후의 21번째 답변 샘플: [5758, 607, 2490, 4160]


In [12]:
print('질문의 최대 길이 :',max(len(question) for question in questions))
print('질문의 평균 길이 :',sum(map(len, questions))/len(questions))
print('답변의 최대 길이 :',max(len(question) for question in questions))
print('답변의 평균 길이 :',sum(map(len, answers))/len(answers))

질문의 최대 길이 : 76
질문의 평균 길이 : 15.0151399813922
답변의 최대 길이 : 76
답변의 평균 길이 : 12.879049310665652


In [13]:
MAX_LENGTH = 20
print(MAX_LENGTH)

20


## 정수 Indexing과 padding을 함수화
-token들에 각각의 index를 부여하게 된다.  
-filtering을 통하여 특정 크기 이하의 문장들을 제거한다.

In [14]:
def tokenize_and_filter(inputs, outputs):
  tokenized_inputs, tokenized_outputs = [], []
  
  for (sentence1, sentence2) in zip(inputs, outputs):
    # 정수 인코딩 과정에서 시작 토큰과 종료 토큰을 추가
    sentence1 = START_TOKEN + tokenizer.encode(sentence1) + END_TOKEN
    sentence2 = START_TOKEN + tokenizer.encode(sentence2) + END_TOKEN

    # 최대 길이 20 이하인 경우에만 데이터셋으로 허용
    if len(sentence1) <= MAX_LENGTH and len(sentence2) <= MAX_LENGTH:
      tokenized_inputs.append(sentence1)
      tokenized_outputs.append(sentence2)
  
  # 최대 길이 20으로 모든 데이터셋을 패딩
  tokenized_inputs = tf.keras.preprocessing.sequence.pad_sequences(
      tokenized_inputs, maxlen=MAX_LENGTH, padding='post')
  tokenized_outputs = tf.keras.preprocessing.sequence.pad_sequences(
      tokenized_outputs, maxlen=MAX_LENGTH, padding='post')
  
  return tokenized_inputs, tokenized_outputs

In [15]:
questions, answers = tokenize_and_filter(questions, answers)
print('단어장의 크기 :',(VOCAB_SIZE))
print('필터링 후의 질문 샘플 개수: {}'.format(len(questions)))
print('필터링 후의 답변 샘플 개수: {}'.format(len(answers)))

단어장의 크기 : 8172
필터링 후의 질문 샘플 개수: 11791
필터링 후의 답변 샘플 개수: 11791


# 3. 모델 구성하기

### 1)교사 학습

언어 모델의 **교사 강요(Teacher Forcing)**는 자연어 처리(NLP) 모델을 훈련할 때 사용하는 중요한 기법 중 하나입니다. 주로 시퀀스 생성 모델(Sequence Generation Models), 예를 들어 **RNN(Recurrent Neural Network)**이나 Transformer 모델을 훈련할 때 사용됩니다.

기본 개념

교사 강요는 모델이 훈련 과정에서 이전에 예측한 값을 기반으로 다음 토큰을 생성하는 대신, **정답 데이터(ground truth)**에서 제공된 실제 값을 사용하여 다음 토큰을 예측하도록 하는 방법입니다. 이를 통해 모델이 빠르게 학습하고, 예측 오류가 누적되는 것을 방지할 수 있습니다.

작동 방식

1. 입력: 모델은 주어진 시퀀스의 첫 번째 단어를 입력받고, 그에 따라 다음 단어를 예측합니다.


2. 교사 강요: 모델이 예측한 단어가 아니라, 실제 정답인 단어를 사용하여 다음 단어를 예측하게 만듭니다.

예: "The cat is"라는 문장이 있을 때, 첫 번째 단어 "The"에 기반해 모델이 "dog"을 예측했더라도, 실제 정답인 "cat"을 다음 입력으로 사용하여 "is"를 예측하게 합니다.



3. 이 과정을 반복하며 전체 시퀀스를 학습합니다.



장점

빠른 수렴: 교사 강요는 모델이 학습하는 동안 매번 올바른 입력을 제공받기 때문에, 모델이 더 빨리 수렴할 수 있습니다.

오차 누적 방지: 모델이 학습 초기에 발생하는 작은 예측 오류가 점점 커지는 문제를 방지합니다.


단점

차이점 학습 문제: 실제 예측 시에는 모델이 이전에 예측한 값들을 기반으로 시퀀스를 생성해야 하는데, 교사 강요는 항상 정답을 기준으로 다음 값을 예측하게 합니다. 이 때문에 훈련 단계와 추론 단계 간의 차이로 인해 모델이 실제 예측 시 성능이 저하될 수 있습니다.

이를 **노출 편향(Exposure Bias)**이라고 합니다.



대안 방법

이 문제를 해결하기 위해 일부 연구에서는 교사 강요 비율을 점진적으로 줄여가는 방법이나, 학습 후반부에는 모델의 예측 값을 사용해 훈련하는 방법 등을 제안하기도 합니다. 예를 들어, Scheduled Sampling은 정답과 모델의 예측값을 일정 비율로 섞어 학습하는 기법입니다.

요약

교사 강요는 모델의 학습 효율을 높이기 위해 정답 데이터를 사용하여 다음 단어를 예측하게 하는 방법으로, 훈련과 추론 간의 차이점을 고려해야 하지만, 기본적으로 효과적인 학습 기법입니다.


In [16]:
BATCH_SIZE = 64
BUFFER_SIZE = 20000

# 디코더는 이전의 target을 다음의 input으로 사용합니다.
# 이에 따라 outputs에서는 START_TOKEN을 제거하겠습니다.
dataset = tf.data.Dataset.from_tensor_slices((
    {
        'inputs': questions,
        'dec_inputs': answers[:, :-1]
    },
    {
        'outputs': answers[:, 1:]
    },
))

dataset = dataset.cache()
dataset = dataset.shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE)
dataset = dataset.prefetch(tf.data.experimental.AUTOTUNE)

### 2) Positional Encoding

In [17]:
# 포지셔널 인코딩 레이어
class PositionalEncoding(tf.keras.layers.Layer):

  def __init__(self, position, d_model):
    super(PositionalEncoding, self).__init__()
    self.pos_encoding = self.positional_encoding(position, d_model)

  def get_angles(self, position, i, d_model):
    angles = 1 / tf.pow(10000, (2 * (i // 2)) / tf.cast(d_model, tf.float32))
    return position * angles

  def positional_encoding(self, position, d_model):
    # 각도 배열 생성
    angle_rads = self.get_angles(
        position=tf.range(position, dtype=tf.float32)[:, tf.newaxis],
        i=tf.range(d_model, dtype=tf.float32)[tf.newaxis, :],
        d_model=d_model)

    # 배열의 짝수 인덱스에는 sin 함수 적용
    sines = tf.math.sin(angle_rads[:, 0::2])
    # 배열의 홀수 인덱스에는 cosine 함수 적용
    cosines = tf.math.cos(angle_rads[:, 1::2])

    # sin과 cosine이 교차되도록 재배열
    pos_encoding = tf.stack([sines, cosines], axis=0)
    pos_encoding = tf.transpose(pos_encoding,[1, 2, 0]) 
    pos_encoding = tf.reshape(pos_encoding, [position, d_model])

    pos_encoding = pos_encoding[tf.newaxis, ...]
    return tf.cast(pos_encoding, tf.float32)

  def call(self, inputs):
    return inputs + self.pos_encoding[:, :tf.shape(inputs)[1], :]


### 3) Attention

In [18]:
# 스케일드 닷 프로덕트 어텐션 함수
def scaled_dot_product_attention(query, key, value, mask):
  # 어텐션 가중치는 Q와 K의 닷 프로덕트
  matmul_qk = tf.matmul(query, key, transpose_b=True)

  # 가중치를 정규화
  depth = tf.cast(tf.shape(key)[-1], tf.float32)
  logits = matmul_qk / tf.math.sqrt(depth)

  # 패딩에 마스크 추가
  if mask is not None:
    logits += (mask * -1e9)

  # softmax적용
  attention_weights = tf.nn.softmax(logits, axis=-1)

  # 최종 어텐션은 가중치와 V의 닷 프로덕트
  output = tf.matmul(attention_weights, value)
  return output

In [19]:
class MultiHeadAttention(tf.keras.layers.Layer):

  def __init__(self, d_model, num_heads, name="multi_head_attention"):
    super(MultiHeadAttention, self).__init__(name=name)
    self.num_heads = num_heads
    self.d_model = d_model

    assert d_model % self.num_heads == 0

    self.depth = d_model // self.num_heads

    self.query_dense = tf.keras.layers.Dense(units=d_model)
    self.key_dense = tf.keras.layers.Dense(units=d_model)
    self.value_dense = tf.keras.layers.Dense(units=d_model)

    self.dense = tf.keras.layers.Dense(units=d_model)

  def split_heads(self, inputs, batch_size):
    inputs = tf.reshape(
        inputs, shape=(batch_size, -1, self.num_heads, self.depth))
    return tf.transpose(inputs, perm=[0, 2, 1, 3])

  def call(self, inputs):
    query, key, value, mask = inputs['query'], inputs['key'], inputs[
        'value'], inputs['mask']
    batch_size = tf.shape(query)[0]

    # Q, K, V에 각각 Dense를 적용합니다
    query = self.query_dense(query)   # 쿼리에 Dense 적용
    key = self.key_dense(key)         # 키에 Dense 적용
    value = self.value_dense(value)   # 값에 Dense 적용

    # 병렬 연산을 위한 머리를 여러 개 만듭니다
    query = self.split_heads(query, batch_size)  # 쿼리를 여러 헤드로 나눔
    key = self.split_heads(key, batch_size)      # 키를 여러 헤드로 나눔
    value = self.split_heads(value, batch_size)  # 값을 여러 헤드로 나눔

    # 스케일드 닷 프로덕트 어텐션 함수
    scaled_attention = scaled_dot_product_attention(query, key, value, mask)

    scaled_attention = tf.transpose(scaled_attention, perm=[0, 2, 1, 3])

    # 어텐션 연산 후에 각 결과를 다시 연결(concatenate)합니다
    concat_attention = tf.reshape(scaled_attention,
                                  (batch_size, -1, self.d_model))

    # 최종 결과에도 Dense를 한 번 더 적용합니다
    outputs = self.dense(concat_attention)

    return outputs

### 4) Padding Mask

In [20]:
def create_padding_mask(x):
  mask = tf.cast(tf.math.equal(x, 0), tf.float32)
  # (batch_size, 1, 1, sequence length)
  return mask[:, tf.newaxis, tf.newaxis, :]

In [21]:
def create_look_ahead_mask(x):
  seq_len = tf.shape(x)[1]
  look_ahead_mask = 1 - tf.linalg.band_part(tf.ones((seq_len, seq_len)), -1, 0)
  padding_mask = create_padding_mask(x)
  return tf.maximum(look_ahead_mask, padding_mask)

### 5) Encoder의 구조

In [22]:
def encoder_layer(units, d_model, num_heads, dropout, name="encoder_layer"):
  inputs = tf.keras.Input(shape=(None, d_model), name="inputs")

  # 패딩 마스크 사용
  padding_mask = tf.keras.Input(shape=(1, 1, None), name="padding_mask")

  # 첫 번째 서브 레이어 : 멀티 헤드 어텐션 수행 (셀프 어텐션)
  attention = MultiHeadAttention(
      d_model, num_heads, name="attention")({
          'query': inputs,
          'key': inputs,
          'value': inputs,
          'mask': padding_mask
      })

  # 어텐션의 결과는 Dropout과 Layer Normalization이라는 훈련을 돕는 테크닉을 수행
  attention = tf.keras.layers.Dropout(rate=dropout)(attention)
  attention = tf.keras.layers.LayerNormalization(
      epsilon=1e-6)(inputs + attention)

  # 두 번째 서브 레이어 : 2개의 완전연결층
  outputs = tf.keras.layers.Dense(units=units, activation='relu')(attention)
  outputs = tf.keras.layers.Dense(units=d_model)(outputs)

  # 완전연결층의 결과는 Dropout과 LayerNormalization이라는 훈련을 돕는 테크닉을 수행
  outputs = tf.keras.layers.Dropout(rate=dropout)(outputs)
  outputs = tf.keras.layers.LayerNormalization(
      epsilon=1e-6)(attention + outputs)

  return tf.keras.Model(
      inputs=[inputs, padding_mask], outputs=outputs, name=name)

In [23]:
def encoder(vocab_size,
            num_layers,
            units,
            d_model,
            num_heads,
            dropout,
            name="encoder"):
  inputs = tf.keras.Input(shape=(None,), name="inputs")

  # 패딩 마스크 사용
  padding_mask = tf.keras.Input(shape=(1, 1, None), name="padding_mask")

  # 임베딩 레이어
  embeddings = tf.keras.layers.Embedding(vocab_size, d_model)(inputs)
  embeddings *= tf.math.sqrt(tf.cast(d_model, tf.float32))

  # 포지셔널 인코딩
  embeddings = PositionalEncoding(vocab_size, d_model)(embeddings)

  outputs = tf.keras.layers.Dropout(rate=dropout)(embeddings)

  # num_layers만큼 쌓아올린 인코더의 층.
  for i in range(num_layers):
    outputs = encoder_layer(
        units=units,
        d_model=d_model,
        num_heads=num_heads,
        dropout=dropout,
        name="encoder_layer_{}".format(i),
    )([outputs, padding_mask])

  return tf.keras.Model(
      inputs=[inputs, padding_mask], outputs=outputs, name=name)

### 6) Decoder의 구조

In [24]:
def decoder_layer(units, d_model, num_heads, dropout, name="decoder_layer"):
  inputs = tf.keras.Input(shape=(None, d_model), name="inputs")
  enc_outputs = tf.keras.Input(shape=(None, d_model), name="encoder_outputs")
  look_ahead_mask = tf.keras.Input(
      shape=(1, None, None), name="look_ahead_mask")
  padding_mask = tf.keras.Input(shape=(1, 1, None), name='padding_mask')

  # 첫 번째 서브 레이어 : 멀티 헤드 어텐션 수행 (셀프 어텐션)
  attention1 = MultiHeadAttention(
      d_model, num_heads, name="attention_1")(inputs={
          'query': inputs,
          'key': inputs,
          'value': inputs,
          'mask': look_ahead_mask
      })

  # 멀티 헤드 어텐션의 결과는 LayerNormalization이라는 훈련을 돕는 테크닉을 수행
  attention1 = tf.keras.layers.LayerNormalization(
      epsilon=1e-6)(attention1 + inputs)

  # 두 번째 서브 레이어 : 마스크드 멀티 헤드 어텐션 수행 (인코더-디코더 어텐션)
  attention2 = MultiHeadAttention(
      d_model, num_heads, name="attention_2")(inputs={
          'query': attention1,
          'key': enc_outputs,
          'value': enc_outputs,
          'mask': padding_mask
      })

  # 마스크드 멀티 헤드 어텐션의 결과는
  # Dropout과 LayerNormalization이라는 훈련을 돕는 테크닉을 수행
  attention2 = tf.keras.layers.Dropout(rate=dropout)(attention2)
  attention2 = tf.keras.layers.LayerNormalization(
      epsilon=1e-6)(attention2 + attention1)

  # 세 번째 서브 레이어 : 2개의 완전연결층
  outputs = tf.keras.layers.Dense(units=units, activation='relu')(attention2)
  outputs = tf.keras.layers.Dense(units=d_model)(outputs)

  # 완전연결층의 결과는 Dropout과 LayerNormalization 수행
  outputs = tf.keras.layers.Dropout(rate=dropout)(outputs)
  outputs = tf.keras.layers.LayerNormalization(
      epsilon=1e-6)(outputs + attention2)

  return tf.keras.Model(
      inputs=[inputs, enc_outputs, look_ahead_mask, padding_mask],
      outputs=outputs,
      name=name)

In [25]:
def decoder(vocab_size,
            num_layers,
            units,
            d_model,
            num_heads,
            dropout,
            name='decoder'):
  inputs = tf.keras.Input(shape=(None,), name='inputs')
  enc_outputs = tf.keras.Input(shape=(None, d_model), name='encoder_outputs')
  look_ahead_mask = tf.keras.Input(
      shape=(1, None, None), name='look_ahead_mask')

  # 패딩 마스크
  padding_mask = tf.keras.Input(shape=(1, 1, None), name='padding_mask')
  
  # 임베딩 레이어
  embeddings = tf.keras.layers.Embedding(vocab_size, d_model)(inputs)
  embeddings *= tf.math.sqrt(tf.cast(d_model, tf.float32))

  # 포지셔널 인코딩
  embeddings = PositionalEncoding(vocab_size, d_model)(embeddings)

  # Dropout이라는 훈련을 돕는 테크닉을 수행
  outputs = tf.keras.layers.Dropout(rate=dropout)(embeddings)

  for i in range(num_layers):
    outputs = decoder_layer(
        units=units,
        d_model=d_model,
        num_heads=num_heads,
        dropout=dropout,
        name='decoder_layer_{}'.format(i),
    )(inputs=[outputs, enc_outputs, look_ahead_mask, padding_mask])

  return tf.keras.Model(
      inputs=[inputs, enc_outputs, look_ahead_mask, padding_mask],
      outputs=outputs,
      name=name)

### 7) Transformer의 구조

In [26]:
def transformer(vocab_size,
                num_layers,
                units,
                d_model,
                num_heads,
                dropout,
                name="transformer"):
  inputs = tf.keras.Input(shape=(None,), name="inputs")
  dec_inputs = tf.keras.Input(shape=(None,), name="dec_inputs")

  # 인코더에서 패딩을 위한 마스크
  enc_padding_mask = tf.keras.layers.Lambda(
      create_padding_mask, output_shape=(1, 1, None),
      name='enc_padding_mask')(inputs)

  # 디코더에서 미래의 토큰을 마스크 하기 위해서 사용합니다.
  # 내부적으로 패딩 마스크도 포함되어져 있습니다.
  look_ahead_mask = tf.keras.layers.Lambda(
      create_look_ahead_mask,
      output_shape=(1, None, None),
      name='look_ahead_mask')(dec_inputs)

  # 두 번째 어텐션 블록에서 인코더의 벡터들을 마스킹
  # 디코더에서 패딩을 위한 마스크
  dec_padding_mask = tf.keras.layers.Lambda(
      create_padding_mask, output_shape=(1, 1, None),
      name='dec_padding_mask')(inputs)

  # 인코더
  enc_outputs = encoder(
      vocab_size=vocab_size,
      num_layers=num_layers,
      units=units,
      d_model=d_model,
      num_heads=num_heads,
      dropout=dropout,
  )(inputs=[inputs, enc_padding_mask])

  # 디코더
  dec_outputs = decoder(
      vocab_size=vocab_size,
      num_layers=num_layers,
      units=units,
      d_model=d_model,
      num_heads=num_heads,
      dropout=dropout,
  )(inputs=[dec_inputs, enc_outputs, look_ahead_mask, dec_padding_mask])

  # 완전연결층
  outputs = tf.keras.layers.Dense(units=vocab_size, name="outputs")(dec_outputs)

  return tf.keras.Model(inputs=[inputs, dec_inputs], outputs=outputs, name=name)

In [27]:
tf.keras.backend.clear_session()

# 하이퍼파라미터
NUM_LAYERS = 2 # 인코더와 디코더의 층의 개수
D_MODEL = 256 # 인코더와 디코더 내부의 입, 출력의 고정 차원
NUM_HEADS = 8 # 멀티 헤드 어텐션에서의 헤드 수 
UNITS = 512 # 피드 포워드 신경망의 은닉층의 크기
DROPOUT = 0.1 # 드롭아웃의 비율

model = transformer(
    vocab_size=VOCAB_SIZE,
    num_layers=NUM_LAYERS,
    units=UNITS,
    d_model=D_MODEL,
    num_heads=NUM_HEADS,
    dropout=DROPOUT)

model.summary()

Model: "transformer"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
inputs (InputLayer)             [(None, None)]       0                                            
__________________________________________________________________________________________________
dec_inputs (InputLayer)         [(None, None)]       0                                            
__________________________________________________________________________________________________
enc_padding_mask (Lambda)       (None, 1, 1, None)   0           inputs[0][0]                     
__________________________________________________________________________________________________
encoder (Functional)            (None, None, 256)    3146240     inputs[0][0]                     
                                                                 enc_padding_mask[0][0] 

### 8) 손실 함수

In [28]:
def loss_function(y_true, y_pred):
  y_true = tf.reshape(y_true, shape=(-1, MAX_LENGTH - 1))
  
  loss = tf.keras.losses.SparseCategoricalCrossentropy(
      from_logits=True, reduction='none')(y_true, y_pred)

  mask = tf.cast(tf.not_equal(y_true, 0), tf.float32)
  loss = tf.multiply(loss, mask)

  return tf.reduce_mean(loss)

### 9) 커스텀 된 학습률(Learning Rate)

이 코드는 학습률 스케줄러를 정의한 것으로, 학습 과정 중 학습률(learning rate)을 동적으로 조정하기 위한 클래스를 구현한 것입니다. 이 스케줄러는 주로 Transformer 모델에서 사용되는 Custom Learning Rate Schedule을 구현한 형태로, 일정 단계 동안 학습률을 서서히 증가시키고 그 이후에는 감소시키는 패턴을 따릅니다.

코드 설명
1. CustomSchedule 클래스:
CustomSchedule 클래스는 tf.keras.optimizers.schedules.LearningRateSchedule을 상속받아, 학습률 스케줄을 정의하는 클래스입니다. 이 클래스는 학습 단계에 따라 학습률을 조절합니다.
__init__ 메서드는 초기화를 담당하며, d_model과 warmup_steps를 인자로 받습니다. d_model은 모델의 차원, warmup_steps는 학습률을 서서히 증가시키는 단계 수를 설정합니다.
2. call 메서드:
이 메서드는 매 학습 스텝에서 호출되며, 해당 학습 스텝에 맞는 학습률을 계산하여 반환합니다.
인자로 받는 step은 현재 학습 단계(스텝)입니다.
3. 학습률 계산 로직:
학습률은 다음 두 가지 요인을 결합하여 계산됩니다:
arg1 = tf.math.rsqrt(step): 현재 스텝의 역 제곱근을 계산합니다. 학습이 진행될수록 이 값은 감소합니다.
arg2 = step * (self.warmup_steps**-1.5): 현재 스텝을 warmup_steps의 -1.5승과 곱합니다. 초기에는 이 값이 작아져서 학습률이 서서히 증가하도록 합니다.
최종 학습률은 arg1과 arg2 중 더 작은 값에 tf.math.rsqrt(self.d_model)를 곱하여 결정됩니다. 즉, 학습 초기에는 학습률이 증가(warmup)하고, 특정 스텝 이후에는 감소하는 패턴을 따릅니다.
4. 학습률의 동작 방식:
Warmup phase: step이 작을 때는 arg2가 지배적이므로 학습률이 선형적으로 증가합니다. 이로 인해 초기 학습에서의 불안정성을 줄일 수 있습니다.
Decay phase: step이 커질수록 arg1이 지배적이 되어 학습률이 점차 감소하게 됩니다. 이로써 학습이 안정되면서 학습률을 줄여 모델이 더 정교하게 학습되도록 합니다.
요약:
이 스케줄러는 학습 초기에는 학습률을 선형적으로 증가시키고(이 과정이 warmup), 이후 학습률을 감소시키는 방식으로 동작합니다.
주로 Transformer와 같은 모델에서 학습 안정성 및 성능을 높이기 위해 사용됩니다.
스케줄링을 통해 모델이 초기 학습률로 인해 급격하게 최적화되지 않도록 하고, 이후 학습이 안정되면 학습률을 줄여 더 세밀한 학습을 할 


Warmup은 딥러닝에서 **학습률(learning rate)**을 초기 학습 단계에서 점진적으로 증가시키는 기법을 말합니다. 이를 통해 모델이 학습 초기의 불안정성을 피하고, 더 안정적으로 최적화될 수 있도록 도와줍니다. Warmup은 주로 학습이 불안정한 복잡한 모델이나, 학습 초기에 큰 학습률을 사용할 경우 성능이 나빠지는 경우에 유용합니다.

Warmup의 필요성
딥러닝 모델을 학습할 때, 학습률은 모델의 성능과 수렴 속도에 큰 영향을 미칩니다. 학습률이 너무 크면 최적화 과정에서 과도하게 큰 업데이트가 이루어져서 모델이 불안정해질 수 있고, 학습률이 너무 작으면 학습이 느려지고 수렴 속도가 늦어질 수 있습니다. 특히 초기 학습 단계에서는 모델이 아직 잘 정돈되지 않은 상태이므로, 큰 학습률로 시작하면 모델이 최적화에서 벗어나거나 성능이 나빠질 수 있습니다.

Warmup의 기본 아이디어
Warmup은 학습 초기에는 낮은 학습률로 학습을 시작하고, 이후 일정한 단계까지 학습률을 점진적으로 증가시키는 방식입니다. 이를 통해 모델이 학습 초기에 안정적으로 최적화되도록 하고, 점차 학습률을 증가시켜 더 빠른 학습이 가능해지도록 합니다.

Warmup의 적용 방식:
초기 학습률: 학습이 시작되면, 매우 낮은 학습률로 모델을 학습시킵니다.
점진적 증가: 학습률을 일정한 단계(warmup steps) 동안 점진적으로 증가시킵니다.
최대 학습률 도달: warmup 단계가 끝나면, 학습률은 설정된 최대 학습률에 도달하게 됩니다.
학습률 조정: 이후에는 학습률이 고정되거나, **학습률 감소(Learning Rate Decay)**를 적용하여 학습률을 줄이는 방식으로 학습을 계속합니다.
Warmup의 예
Transformer와 같은 대규모 모델에서 warmup은 자주 사용됩니다. 특히 초기 단계에서 학습률을 천천히 증가시키고, 이후 학습률을 줄이는 방식이 안정적인 학습을 가능하게 합니다. 이를 통해 모델이 수렴 과정에서 큰 폭으로 요동치지 않도록 돕습니다.

Warmup의 장점
학습 초기의 불안정성 완화: 초기 학습에서 너무 큰 학습률을 사용하는 경우 모델이 불안정해지거나, 최적화가 비효율적으로 이루어질 수 있습니다. Warmup은 이 문제를 완화해줍니다.
더 안정적이고 효과적인 최적화: Warmup을 통해 모델이 학습 초기에 작은 학습률로 안정적인 학습을 하다가, 충분히 안정화된 후에 학습률을 크게 하여 빠르게 학습할 수 있습니다.
복잡한 모델에 유리: 대규모 모델이나 복잡한 데이터셋을 다룰 때, Warmup은 모델이 더 안정적으로 학습할 수 있도록 도와줍니다.
Warmup이 자주 사용되는 사례
Transformer 모델: 자연어 처리(NLP)에서 Transformer 구조는 초기 학습에서 Warmup을 사용하는데, 이는 매우 큰 모델의 복잡성을 다루기 위함입니다.
대규모 신경망: 매우 깊거나 복잡한 신경망에서 학습 초기에 학습률을 점진적으로 증가시키는 것이 학습의 안정성과 성능에 유리합니다.
요약
Warmup은 학습 초기에는 낮은 학습률로 시작하여 점진적으로 학습률을 증가시키는 방법으로, 초기 학습 불안정성을 줄이고 최적화 성능을 높일 수 있습니다. 특히 복잡한 모델이나 대규모 데이터셋을 다룰 때 유용하며, Transformer와 같은 모델에서 자주 사용됩니다.

In [29]:
class CustomSchedule(tf.keras.optimizers.schedules.LearningRateSchedule):

  def __init__(self, d_model, warmup_steps=4000):
    super(CustomSchedule, self).__init__()

    self.d_model = d_model
    self.d_model = tf.cast(self.d_model, tf.float32)

    self.warmup_steps = warmup_steps

  def __call__(self, step):
    arg1 = tf.math.rsqrt(step)
    arg2 = step * (self.warmup_steps**-1.5)

    return tf.math.rsqrt(self.d_model) * tf.math.minimum(arg1, arg2)

### 10) 모델 컴파일

In [30]:
learning_rate = CustomSchedule(D_MODEL)

optimizer = tf.keras.optimizers.Adam(
    learning_rate, beta_1=0.9, beta_2=0.98, epsilon=1e-9)

def accuracy(y_true, y_pred):
  y_true = tf.reshape(y_true, shape=(-1, MAX_LENGTH - 1))
  return tf.keras.metrics.sparse_categorical_accuracy(y_true, y_pred)

model.compile(optimizer=optimizer, loss=loss_function, metrics=[accuracy])

In [31]:
EPOCHS = 100
model.fit(dataset, epochs=EPOCHS, verbose=1)

Epoch 1/100
185/185 [==============================] - 13s 37ms/step - loss: 2.8997 - accuracy: 0.0401
Epoch 2/100
185/185 [==============================] - 7s 37ms/step - loss: 2.4997 - accuracy: 0.0526
Epoch 3/100
185/185 [==============================] - 7s 37ms/step - loss: 2.2782 - accuracy: 0.0548
Epoch 4/100
185/185 [==============================] - 7s 37ms/step - loss: 2.1598 - accuracy: 0.0594
Epoch 5/100
185/185 [==============================] - 7s 37ms/step - loss: 2.0606 - accuracy: 0.0659
Epoch 6/100
185/185 [==============================] - 7s 38ms/step - loss: 1.9559 - accuracy: 0.0724
Epoch 7/100
185/185 [==============================] - 7s 37ms/step - loss: 1.8440 - accuracy: 0.0792
Epoch 8/100
185/185 [==============================] - 7s 37ms/step - loss: 1.7218 - accuracy: 0.0875
Epoch 9/100
185/185 [==============================] - 7s 37ms/step - loss: 1.5870 - accuracy: 0.0983
Epoch 10/100
185/185 [==============================] - 7s 37ms/step - loss: 1.43

# 4. 모델 평가하기

In [37]:
def decoder_inference(sentence):
  sentence = preprocess_sentence2(sentence)

  # 입력된 문장을 정수 인코딩 후, 시작 토큰과 종료 토큰을 앞뒤로 추가.
  # ex) Where have you been? → [[8331   86   30    5 1059    7 8332]]
  sentence = tf.expand_dims(
      START_TOKEN + tokenizer.encode(sentence) + END_TOKEN, axis=0)

  # 디코더의 현재까지의 예측한 출력 시퀀스가 지속적으로 저장되는 변수.
  # 처음에는 예측한 내용이 없음으로 시작 토큰만 별도 저장. ex) 8331
  output_sequence = tf.expand_dims(START_TOKEN, 0)

  # 디코더의 인퍼런스 단계
  for i in range(MAX_LENGTH):
    # 디코더는 최대 MAX_LENGTH의 길이만큼 다음 단어 예측을 반복합니다.
    predictions = model(inputs=[sentence, output_sequence], training=False)
    predictions = predictions[:, -1:, :]

    # 현재 예측한 단어의 정수
    predicted_id = tf.cast(tf.argma x(predictions, axis=-1), tf.int32)

    # 만약 현재 예측한 단어가 종료 토큰이라면 for문을 종료
    if tf.equal(predicted_id, END_TOKEN[0]):
      break

    # 예측한 단어들은 지속적으로 output_sequence에 추가됩니다.
    # 이 output_sequence는 다시 디코더의 입력이 됩니다.
    output_sequence = tf.concat([output_sequence, predicted_id], axis=-1)

  return tf.squeeze(output_sequence, axis=0)

In [38]:
def sentence_generation(sentence):
  # 입력 문장에 대해서 디코더를 동작 시켜 예측된 정수 시퀀스를 리턴받습니다.
  prediction = decoder_inference(sentence)

  # 정수 시퀀스를 다시 텍스트 시퀀스로 변환합니다.
  predicted_sentence = tokenizer.decode(
      [i for i in prediction if i < tokenizer.vocab_size])

  print('입력 : {}'.format(sentence))
  print('출력 : {}'.format(predicted_sentence))

  return predicted_sentence

In [39]:
sentence_generation('?')

입력 : 1 더하기 1은?
출력 : 짝남이 내가 좋아하는 거 알아버림.


'짝남이 내가 좋아하는 거 알아버림.'